7.22 Dota version

Dota skills json file

In [531]:
import pandas as pd

In [532]:
import itertools

In [533]:
import urllib.request
import requests
from bs4 import BeautifulSoup

In [534]:
import urllib.request
import json
from urllib.request import urlopen

content = urlopen("https://raw.githubusercontent.com/odota/dotaconstants/master/build/abilities.json")

with urlopen("https://raw.githubusercontent.com/odota/dotaconstants/master/build/abilities.json") as json_file:
    data = json.load(json_file)

In [535]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [652]:
with urlopen('https://raw.githubusercontent.com/dqa611/Dota2_analysis/master/dota2_heroes.csv') as csv_file:
    heroes_name_fixed = pd.read_csv(csv_file)

In [653]:
heroes_name_list = heroes_name_fixed['Heroes'].tolist()

In [538]:
[value for key, value in data.items() if 'slardar' in key.lower()]

[{'dname': 'Guardian Sprint',
  'behavior': ['No Target', 'Instant Cast', None],
  'desc': 'Slardar slithers ahead, moving significantly faster and passing through units. Slardar has unlocked max speed and grants movement speed while in the river.',
  'attrib': [{'key': 'bonus_speed',
    'header': 'BONUS MOVE SPEED:',
    'value': ['20%', '28%', '36%', '44%']},
   {'key': 'duration', 'header': 'DURATION:', 'value': '10'},
   {'key': 'river_speed',
    'header': 'RIVER SPEED:',
    'value': ['5', '20', '35', '50'],
    'generated': True},
   {'key': 'puddle_regen', 'header': 'SCEPTER HP REGEN:', 'value': '25'},
   {'key': 'puddle_armor', 'header': 'SCEPTER BONUS ARMOR:', 'value': '12'},
   {'key': 'puddle_status_resistance',
    'header': 'SCEPTER STATUS RESISTANCE:',
    'value': '40%'},
   {'key': 'river_speed_tooltip',
    'header': 'BONUS MOVE SPEED IN RIVER:',
    'value': ['25%', '48%', '66%', '94%']}],
  'cd': '17',
  'img': '/apps/dota2/images/abilities/slardar_sprint_md.png'},

In [539]:
# remove duplicates and keep the list order 
# http://www.martinbroadhurst.com/removing-duplicates-from-a-list-while-preserving-order-in-python.html

def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

what im trying to do so far is to websrap dota page theory about hero counters. 

In theory, each heroes will have counter heroes and items

In [540]:
def get_heroes_matchup_info(hero_name, heroes_mentioned):
    
    # Part 1: clean up the information collected from the webite  
    # filter non type element in the list
    heroes_mentioned = list(filter(None, heroes_mentioned)) 

    # cut short the list to only relevant heroes
    heroes_mentioned_clean = heroes_mentioned[heroes_mentioned.index('#Good_against...')+2 : 
                                              heroes_mentioned.index('/Special:Categories') -1]

    # remove '/' i the list
    heroes_mentioned_clean = [heroes_mentioned[i].strip('/') for i in 
                              range(len(heroes_mentioned))]    

    # remove duplicates in the list    
    heroes_mentioned_clean = unique(heroes_mentioned_clean)

    # remove skills and items (can think more about this later)
    heroes_mentioned_clean1 = []

    for i in range(len(heroes_mentioned_clean)):
        if '#' not in heroes_mentioned_clean[i]:
            heroes_mentioned_clean1.append(heroes_mentioned_clean[i])

    # filter only heroes        
    heroes_mentioned_clean2 = [heroes_mentioned_clean1[i] for i in 
                               range(len(heroes_mentioned_clean1)) if 
                               heroes_mentioned_clean1[i] in heroes_name_list]

    # Part 2: populate the hero's matchup info   
    # In the page, heroes that counters the hero well is in red  
    heroes_countered1 = []

    productDivs = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px red;'})
    for div in productDivs:
        heroes_countered1.append(div.find('a')['href'].strip('/'))

    # In the page, heroes that the hero counters well is in green       
    heroes_counter1 = []

    productDivs_chartreuse = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px chartreuse;'})
    productDivs_green = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px green;'})
    productDivs = productDivs_chartreuse + productDivs_green
    for div in productDivs:
        heroes_counter1.append(div.find('a')['href'].strip('/'))

    # In the page, heroes good with are list in blue     
    heroes_good_with1 = []

    productDivs = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px skyblue;'})
    for div in productDivs:
        heroes_good_with1.append(div.find('a')['href'].strip('/'))

    # In the page, great countered heroes are mentioned in red, then there are some other heroes mentioned before the page moves on to the next section
    # last hero mentioned in red 
    last_red = heroes_mentioned_clean2.index(heroes_countered1[-1])
    # first hero mentioned in green
    first_green = heroes_mentioned_clean2.index(heroes_counter1[0])
    # last hero mentioned in green
    last_green = heroes_mentioned_clean2.index(heroes_counter1[-1])

    # first hero mentioned in blue
    first_blue = heroes_mentioned_clean2.index(heroes_good_with1[0])
    # last hero mentioned in blue
    last_blue = heroes_mentioned_clean2.index(heroes_good_with1[-1])

    # Create list of heroes matchups    
    # Create countered level 2
    if last_red+1 == first_green:
        heroes_countered2 = []
    else:     
        heroes_countered2 = heroes_mentioned_clean2[last_red+1 : first_green]

    # Create counter level 2
    if last_green+1 == first_blue: 
        heroes_counter2 = []
    else:
        heroes_counter2 = heroes_mentioned_clean2[last_green+1 : first_blue]

    # Create good with level 2
    heroes_good_with2 = heroes_mentioned_clean2[last_blue+1 :]

    # Part 3: export the info 
    # to an object
    heroes_matchup_info = {'heroes_countered1': heroes_countered1, 
                           'heroes_countered2': heroes_countered2,
                           'heroes_counter1': heroes_counter1, 
                           'heroes_counter2': heroes_counter2,
                           'heroes_good_with1': heroes_good_with1,
                           'heroes_good_with2': heroes_good_with2}
    
    return heroes_matchup_info

get all the link in gamepedia page

In [541]:
r = requests.get('https://dota2.gamepedia.com/Matchups')
soup = BeautifulSoup(r.text)

base_url = 'https://dota2.gamepedia.com/Matchups'
heroes_counters_link = []

for link in soup.find_all('a'):
    url = link.get('href')
    if url and '/Counters' in url:
        heroes_counters_link.append(url)
        
heroes_counters_link = list(sorted(set(heroes_counters_link)))

In [542]:
heroes_matchup_dict = {}

for i in range(len(heroes_counters_link)):
    r = requests.get('https://dota2.gamepedia.com' + heroes_counters_link[i])
    soup = BeautifulSoup(r.text)

    hero_name = heroes_counters_link[i].strip('/')[:-9]

    heroes_mentioned = []
    for link in soup.findAll('a'):
        heroes_mentioned.append(link.get('href')) 
    
    heroes_matchup_dict[hero_name] = get_heroes_matchup_info(hero_name, heroes_mentioned)

list of all heroes and items are mentioned in the counter page of the hero, we need to rank great against, normal against, item against and skills against

In [543]:
heroes_matchup_pair = list(itertools.permutations(heroes_name_list,2))

Matchup Score

In [568]:
matchup_score = []
for i in range(len(heroes_matchup_pair)):    
    matchup_score.append(list([heroes_matchup_pair[i],0]))

In [574]:
for i in range(len(heroes_matchup_pair)):

    hero1_in_pair = heroes_matchup_pair[i][0]
    hero2_in_pair = heroes_matchup_pair[i][1]
    
    list_counter_countered = list(itertools.chain(*[heroes_matchup_dict[hero1_in_pair]['heroes_countered2'],
                                                    heroes_matchup_dict[hero1_in_pair]['heroes_counter2'],
                                                    heroes_matchup_dict[hero1_in_pair]['heroes_countered1'],
                                                    heroes_matchup_dict[hero1_in_pair]['heroes_counter1']]
                                                 ))

# 0 means the 2 heores have ways to counter each other
# negative means hero 1 in the pair is countered by hero 2
# positive means hero 1 in the pair counters hero 2

    if hero2_in_pair not in list_counter_countered:
        matchup_score[i][1] = 0
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_countered2']:
        matchup_score[i][1] = -.5
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_counter2']:
        matchup_score[i][1] = .5
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_countered1']:
        matchup_score[i][1] = -1
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_counter1']:
        matchup_score[i][1] = 1

In [577]:
matchup_score1 = matchup_score[:]

for i in range(len(heroes_matchup_pair_add_score)):

    hero1_in_pair = heroes_matchup_pair[i][0]
    hero2_in_pair = heroes_matchup_pair[i][1]

    similar_pairs = [y for y in matchup_score if hero1_in_pair in y[0] and hero2_in_pair in y[0]]    

    heroes_pair_1 = similar_pairs[0]
    heroes_pair_2 = similar_pairs[1]

    if abs(heroes_pair_1[1]) > abs(heroes_pair_2[1]):
        matchup_score1[matchup_score1.index(heroes_pair_2)][1] = -heroes_pair_1[1]
    elif abs(heroes_pair_1[1]) < abs(heroes_pair_2[1]):
        matchup_score1[matchup_score1.index(heroes_pair_1)][1] = -heroes_pair_2[1]

Good With Score

In [613]:
good_with_score = []
for i in range(len(heroes_matchup_pair)):    
    good_with_score.append(list([heroes_matchup_pair[i],0]))

In [625]:
for i in range(len(heroes_matchup_pair)):

    hero1_in_pair = heroes_matchup_pair[i][0]
    hero2_in_pair = heroes_matchup_pair[i][1]
    
    list_good_with = list(itertools.chain(*[heroes_matchup_dict[hero1_in_pair]['heroes_good_with1'],
                                                    heroes_matchup_dict[hero1_in_pair]['heroes_good_with2']
                                           ]))

# 0 means the 2 heroes don't significantly go well together
# 0.5 means OK
# 1 means combo buddy                                        

    if hero2_in_pair not in list_good_with:
        good_with_score[i][1] = 0
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_good_with2']:
        good_with_score[i][1] = .5
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_good_with1']:
        good_with_score[i][1] = 1

In [627]:
good_with_score1 = good_with_score[:]

for i in range(len(heroes_matchup_pair_add_score)):

    hero1_in_pair = heroes_matchup_pair[i][0]
    hero2_in_pair = heroes_matchup_pair[i][1]

    similar_pairs = [y for y in good_with_score if hero1_in_pair in y[0] and hero2_in_pair in y[0]]    

    heroes_pair_1 = similar_pairs[0]
    heroes_pair_2 = similar_pairs[1]

    if abs(heroes_pair_1[1]) > abs(heroes_pair_2[1]):
        good_with_score1[good_with_score1.index(heroes_pair_2)][1] = heroes_pair_1[1]
    elif abs(heroes_pair_1[1]) < abs(heroes_pair_2[1]):
        good_with_score1[good_with_score1.index(heroes_pair_1)][1] = heroes_pair_2[1]

the next step is to have a 5v5 match. Each radiant hero will be matchup to the each hero in dire. Add the total sum each, then add all 5. 
Another factor will be heroes good at. Each good one will be add 1 or 0.5 points. 

Each hero in Radiant will be scored against each hero in Dire. Their scores are added up together. If the total is a positive number, it means in total, Radiant heroes individually counters Dire heroes

In [ ]:
Input Radiant Team

In [700]:
radiant_hero_list_raw = input('Insert Radiant Heroes here (by shortcut name). Seaparate by comma: ')
hero_name_process(radiant_hero_list_raw)

Insert Radiant Heroes here (by shortcut name). Seaparate by comma: cm, axe, clinkz


['Crystal_Maiden', 'Axe', 'Clinkz']

In [699]:
def hero_name_process(hero_list_raw):
    hero_list_raw = hero_list_raw.split(',')
    hero_list_raw1 = [str(item).strip() for item in hero_list_raw]

    hero_list = []

    for i in range(len(hero_list_raw)):
        hero_list.append(heroes_name_fixed[heroes_name_fixed['key'] == hero_list_raw1[i]]['Heroes'].item())
        
    return hero_list

In [ ]:
Input Dire Team

In [701]:
dire_hero_list_raw = input('Insert dire Heroes here (by shortcut name). Seaparate by comma: ')
hero_name_process(dire_hero_list_raw)

Insert dire Heroes here (by shortcut name). Seaparate by comma: jugg, lion, od


['Juggernaut', 'Lion', 'Outworld_Devourer']

In [690]:
radiant_team = [hero for hero in radiant_hero_list]
dire_team = [hero for hero in dire_hero_list]

radiant_counter_score = 0 

for hero_radiant in radiant_team:
    for hero_dire in dire_team:
        radiant_counter_score += [y for y in matchup_score1 if hero_radiant in y[0][0] and hero_dire in y[0][1]][0][1]
    
radiant_counter_score

-2.5

5 Radiant heroes good_with score will be added pair by pair to total the Team synergy score

In [635]:
radiant_team_pair = list(itertools.combinations(radiant_hero_list,2))
radiant_good_with_score = 0

for i in range(len(radiant_team_pair)):
    radiant_good_with_score += [y for y in good_with_score1 if \
                                radiant_team_pair[i][0] in y[0][0] and \
                                radiant_team_pair[i][1] in y[0][1]][0][1]
radiant_good_with_score

3.5

Same thing for 5 Dire heroes, but at Radiant's pespective this a bad thing for Radiant

In [643]:
dire_team_pair = list(itertools.combinations(dire_hero_list,2))
dire_good_with_score = 0

for i in range(len(dire_team_pair)):
    dire_good_with_score += [y for y in good_with_score1 if \
                                dire_team_pair[i][0] in y[0][0] and \
                                dire_team_pair[i][1] in y[0][1]][0][1]
dire_good_with_score

1.5

In [640]:
radiant_total_score = radiant_counter_score + radiant_good_with_score - dire_good_with_score
print('Radiant Team Score vs. Dire Team is: ' + str(radiant_total_score))

Radiant Team Score vs. Dire Team is: -0.5


Ban mode:

RB = Radiant Ban
DB = Dire Ban
RP = Radiant Pick
DP = Dire Pick

RB, DB, RB, DB, RB, DB, 

RP, DP, DP, RP, 

RB, DB, RB, DB, 

RP, DP, RP, DP, 

DB, RB, 

RP, DP

In [733]:
def discard_pool(discard_pool):
    remaining_pool = [value for value in heroes_name_list if value not in discard_pool]
    return remaining_pool

1st Ban Phase

In [747]:
discard_pool = []
RB1 = input('Radiant 1st ban: ')
RB1 = hero_name_process(RB1)
discard_pool.append(RB1)
print(RB1)

Radiant 1st ban: cm
['Crystal_Maiden']


In [751]:
DB1 = input('Dire 1st ban: ')
DB1 = hero_name_process(DB1)
if DB1 in discard_pool:
    print('This hero has already been selected')
else:
    discard_pool.append(DB1)
    print(DB1)

Dire 1st ban: ursa
['Ursa']


In [752]:
RB2 = input('Radiant 2nd ban: ')
RB2 = hero_name_process(RB2)
if RB2 in discard_pool:
    print('This hero has already been selected')
else:
    discard_pool.append(RB2)
    print(RB2)    

Radiant 2nd ban: lion
['Lion']


In [755]:
DB2 = input('Dire 2nd ban: ')
DB2 = hero_name_process(DB2)
if DB2 in discard_pool:
    print('This hero has already been selected')
else:
    discard_pool.append(DB2)
    print(DB2) 

Dire 2nd ban: ds
['Dark_Seer']


In [756]:
RB3 = input('Radiant 3rd ban: ')
RB3 = hero_name_process(RB3)
if RB3 in discard_pool:
    print('This hero has already been selected')
else:
    discard_pool.append(RB2)
    print(RB3) 

Radiant 3rd ban: invoker
['Invoker']


In [757]:
DB3 = input('Dire 3rd ban: ')
DB3 = hero_name_process(DB3)
if DB3 in discard_pool:
    print('This hero has already been selected')
else:
    discard_pool.append(DB3)
    print(DB3) 

Dire 3rd ban: tinker
['Tinker']


In [760]:
first_ban_list = list(itertools.chain(*[RB1, DB1, RB2, DB2, RB3, DB3]))
first_ban_list

['Crystal_Maiden', 'Ursa', 'Lion', 'Dark_Seer', 'Invoker', 'Tinker']